In [124]:
import requests
from env import env
import urllib.parse
from datetime import datetime

In [120]:
csUrl = "https://www.cheapshark.com/api/1.0/"

def supCheapSharkExtractor(path,**kwargs) -> list:

    send = csUrl + path

    for key,value in kwargs.items():
        if value != None: send += f"{key}={value}&"
    return requests.get(send).json()

def getGameByTheme(title:str = None,size:int = 60) -> list:
    """
    Retorna uma lista de jogos baseados em um tema.
    """
    if title: title = urllib.parse.quote(title)
    response = supCheapSharkExtractor(path="games?",title=title,size=size)
    response_id = []

    for i in response: 
        if i['steamAppID'] != None: 
            response_id.append(f"{i['external']} - {i['steamAppID']}")

    return response_id

In [74]:
stUrl = "https://store.steampowered.com/api/"

def supSteamPoweredExtractor(path,**kwargs) -> dict:
    send = stUrl + path

    for key,value in kwargs.items():
        if value != None: send += f"{key}={value}&"

    return requests.get(send).json()

def getGameOverview(appids:str = None):
    response = supSteamPoweredExtractor(path="appdetails?",appids=appids,cc="pt-br",l="portuguese")

    desciption_v = response[appids]["data"]['detailed_description']

    dev_v = "Desenvolvido pela: " + response[appids]["data"]['developers'][0]
    pub_v = "Distribuído pela: " + response[appids]["data"]['publishers'][0]

    price_v = "Preço na steam: "+ response[appids]["data"]['price_overview']['final_formatted']

    genre_v = response[appids]['data']['genres'][0]['description'] # more tests

    date_v = response[appids]['data']['release_date']['date']

    try:
        meta_v = "Metacritic: " + str(response[appids]["data"]['metacritic']['score'])
    except Exception as e:
        meta_v = "Metacritic não disponível"

    try:
        recomendations_v = "Número de avaliações na steam: "+str(response[appids]["data"]["recommendations"]['total'])
    except:
        recomendations_v = "Produto não avaliado"

    min_rec_v = "Requisitos mínimos: " + response[appids]['data']['pc_requirements']['minimum']

    max_rec_v = "Requisitos recomendados: " + response[appids]['data']['pc_requirements']['recommended']


    struct = {
        "desc": desciption_v,
        "dev": dev_v,
        "pub": pub_v,
        "price":price_v,
        "genre":genre_v,
        "date":date_v,
        "meta":meta_v,
        "recomendations":recomendations_v,
        "min_rec":min_rec_v,
        "max_rec":max_rec_v
    }

    return struct

In [122]:
adUrl = "https://api.isthereanydeal.com/"

def supIsThereAnyDealExtractor(path,**kwargs) -> list:

    send = adUrl + path

    for key,value in kwargs.items():
        if value != None: send += f"{key}={value}&"

    return requests.get(send).json()

def getGameLowest(gameid:str = None):
    response = supIsThereAnyDealExtractor(path="v01/game/overview/?",key=env("AnyDealKey"),country="BR",shop="steam",ids=f"app/{gameid}")

    print(type(response['data'][f"app/{gameid}"]))

    try:

        store_today = response['data'][f"app/{gameid}"]['price']['store']
        price_today = response['data'][f"app/{gameid}"]['price']['price_formatted']

    except:
        store_today = None
        price_today = None

    store_hist = response['data'][f"app/{gameid}"]['lowest']['store']
    price_hist = response['data'][f"app/{gameid}"]['lowest']['price_formatted']


    price_url = response['data'][f"app/{gameid}"]['urls']['history']

    data_formatada = datetime.utcfromtimestamp(response['data'][f"app/{gameid}"]['lowest']['recorded']).strftime('%d-%m-%Y') # dia do menor preço histórico

    struct = {
        "store_today":store_today,
        "price_today":price_today,
        "store_hist":store_hist,
        "price_hist":price_hist,
        "price_url":price_url,
        "date":data_formatada
    }

    return struct

def getGamePlains(gameid:str = None):
    response = supIsThereAnyDealExtractor(path="v01/game/plain/id/?",key=env("AnyDealKey"),shop="steam",ids=f"app/{gameid}")
    return response['data'][f"app/{gameid}"]

def getGamePrices(gamePlain:str = None):
    response = supIsThereAnyDealExtractor(path="v01/game/prices/?",key=env("AnyDealKey"),plains=gamePlain,country="BR")

    price_list = []

    try:
        for i in response['data'][response]['list']:
            f"{i['shop']['name']}: {i['price_old']} -> {i['price_new']} ({i['price_cut']}%)"
            price_list.append()
    except:
        return []

    return price_list


## Escolha do tema e jogo

In [ ]:
response = getGameByTheme(title="observation")
for i in response:
    print(i)

## Retorno do overview

In [107]:
gameid = "2150950"
game_overview = getGameOverview(appids=gameid)

In [ ]:
#print(game_overview[gameid]['success'])

#print(game_overview[gameid]["data"]['is_free'])

print(game_overview[gameid]["data"]['detailed_description']) #

print("Desenvolvido pela: " + game_overview[gameid]["data"]['developers'][0]) # 
print("Distribuído pela: " + game_overview[gameid]["data"]['publishers'][0]) #

print("Preço na steam: "+game_overview[gameid]["data"]['price_overview']['final_formatted']) #


print(game_overview[gameid]['data']['genres'][0]['description']) #

print(game_overview[gameid]['data']['release_date']['date']) #

try:

    print("Metacritic: " + str(game_overview[gameid]["data"]['metacritic']['score'])) #

except Exception as e:
    print("Metacritic não disponível") #

try:
    print("Número de avaliações na steam: "+str(game_overview[gameid]["data"]["recommendations"]['total'])) #
except:
    print("Produto não avaliado") #

print("Requisitos mínimos: " + game_overview[gameid]['data']['pc_requirements']['minimum'])

print("Requisitos recomendados: " + game_overview[gameid]['data']['pc_requirements']['recommended'])

#game_overview[gameid]['data']

## Retorno menor preço atual e histórico

In [ ]:
game_lowest = getGameLowest(gameid)
game_lowest

In [ ]:
print(type(game_lowest['data'][f"app/{gameid}"]))

try:

    print(game_lowest['data'][f"app/{gameid}"]['price']['store']) # today´s lowest
    print(game_lowest['data'][f"app/{gameid}"]['price']['price_formatted'])

except:
    print("today == historical")

print(game_lowest['data'][f"app/{gameid}"]['lowest']['store']) # historical lowest
print(game_lowest['data'][f"app/{gameid}"]['lowest']['price_formatted'])


print(game_lowest['data'][f"app/{gameid}"]['urls']['history']) # price history site

print(game_lowest['data'][f"app/{gameid}"]['lowest']['recorded'])

data_formatada = datetime.utcfromtimestamp(game_lowest['data'][f"app/{gameid}"]['lowest']['recorded']).strftime('%d-%m-%Y') # dia do menor preço histórico

print(data_formatada)

## Retorno todos os preços do jogo

In [ ]:
plainResponse = getGamePlains(gameid=gameid)

game_plain = plainResponse['data'][f"app/{gameid}"]

print(game_plain)

pricesResponse = getGamePrices(game_plain)

In [ ]:
#pricesResponse['data'][game_plain]['list']
print(pricesResponse)

price_list = pricesResponse['data'][game_plain]['list']
try:
    if price_list == []:
        print("sem lista")
    else:
        for i in pricesResponse['data'][game_plain]['list']:
            print(i['price_old'],i['price_new'],str(i['price_cut'])+"%",i['shop']['name'])
except Exception as e:
    print("sem lista")